In [ ]:
%run "0a_Workspace_setup.ipynb"

from NHM_helpers.NHM_helpers import (
    hrus_by_poi,
    hrus_by_seg,
    subset_stream_network,
    create_poi_group,
)
from NHM_helpers.map_template import *
from NHM_helpers.NHM_Assist_utilities import make_plots_par_vals, make_HW_cal_level_files

from NHM_helpers.NHM_output_visualization import (
    retrieve_hru_output_info,
    create_sum_var_dataarrays,
    create_mean_var_dataarrays,
    create_sum_var_annual_gdf,
    create_sum_var_annual_df,
    create_sum_var_monthly_df,
    create_var_daily_df,
    create_var_ts_for_poi_basin_df,
    create_sum_seg_var_dataarrays,
    create_streamflow_obs_datasets,
)
from NHM_helpers.output_plots import *

from ast import literal_eval

poi_id_sel = None

crs = 4326

In [ ]:
import matplotlib as mplib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import folium
from folium import Choropleth, Circle, Marker

from folium import plugins
from shapely.geometry import Polygon
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from shapely.geometry import Polygon

from folium.plugins import MarkerCluster


from tobler.dasymetric import masked_area_interpolate
from tobler.model import glm
from tobler.area_weighted import area_interpolate

from libpysal.examples import load_example

import hydroeval as he

# import hyswap
# from hyswap.percentiles import calculate_variable_percentile_thresholds_by_day
# from hyswap.cumulative import calculate_daily_cumulative_values
import calendar
import statistics
from sklearn.metrics import r2_score

In [ ]:
# Create dataframes for hydrofabric elements
hru_gdf, hru_cal_level_txt = create_hru_gdf(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

HW_basins_gdf, HW_basins = make_HW_cal_level_files(hru_gdf)

con.print(
    f"{workspace_txt}",
    f"\n{hru_cal_level_txt}",
)

# Retrieve pywatershed output file information for mapping and plotting
plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)

#### Compute KGE for all gages to color the icon on the map

In [ ]:
# Read in simulated flows
output_var_sel = "seg_outflow"
var_daily, sum_var_monthly, sum_var_annual, var_units, var_desc = (
    create_sum_seg_var_dataarrays(
        out_dir,
        output_var_sel,
        plot_start_date,
        plot_end_date,
        water_years,
    )
)

In [ ]:
# Read in observed flows
# Note that the model start and stop times in the control file should be the same as the observation start and stop times.

poi_name_df, obs, obs_efc, obs_annual = create_streamflow_obs_datasets(
    output_netcdf_filename,
    plot_start_date,
    plot_end_date,
    water_years,
)

In [ ]:
# Add monthly KGE to poi_df
poi_df = calculate_monthly_kge_in_poi_df(
    obs,
    var_daily,
    poi_df,
)

# Create an interactive map to evaluate streamflow at poi_gages

In [ ]:
from IPython.display import display
from folium.plugins import MeasureControl
from folium.utilities import Element
from folium.plugins import FloatImage
import base64


# Load standard map settings
pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(
    hru_gdf, poi_df, poi_id_sel
)

USGSHydroCached_layer, USGStopo_layer, Esri_WorldImagery, OpenTopoMap = (
    folium_map_tiles()
)

minimap = create_minimap()

# Clear map if previously created
m = folium.Map()

# Create map
m = folium.Map(
    location=[pfile_lat, pfile_lon],
    # width=1000, height=600,
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    control_scale=True,
)

# Add base map layers
USGStopo_layer.add_to(m)
OpenTopoMap.add_to(m)
Esri_WorldImagery.add_to(m)

# Add widgets
m.add_child(minimap)
m.add_child(MeasureControl(position="bottomright"))

# Create and add hru calibration levels (colors)
hru_cal_map = folium.GeoJson(
    HW_basins_gdf,  # hru_gdf_map,
    style_function=cal_style_function,
    # highlight_function = highlight_function_hru_map,
    name="HRU cal level",
    z_index_offset=40002,
).add_to(m)

# Create and add hru boundaries and data
hru_map = create_hru_map(hru_gdf)
tooltip_hru = folium.GeoJsonPopup(
    fields=["hw_id"], aliases=["Headwater id"], labels=True
)
hru_map.add_child(tooltip_hru)
hru_map.add_to(m)

# Create and add headwater basin boundaries
hw_basins_map = folium.GeoJson(
    HW_basins, style_function=hw_basin_style, name="HW basin boundary"
).add_to(m)

# Create/Add segment map
seg_map = create_segment_map_hide(seg_gdf)
seg_map.add_to(m)

# create and add POI marker clusters (marker and label)
marker_cluster, marker_cluster_label_poi = create_streamflow_poi_markers(
    poi_df,
    Folium_maps_dir,
    output_var_sel,
)
marker_cluster.add_to(m)
marker_cluster_label_poi.add_to(m)

plugins.Fullscreen(position="topleft").add_to(m)
folium.LayerControl(collapsed=True, position="bottomright", autoZIndex=True).add_to(m)


explan_txt = f"Gage icon color indicates monthly KGE value: red<0.5 , 0.5<=yellow<0.7,  green>=0.7<br>All HWs (oulined red) were calibrated with statistical targets at the HW outlet (yellow);<br> some HWs were further calibrated with streamflow observations (green) at selected gages (bold outlined)."
title_html = f"<h1 style='position:absolute;z-index:100000;font-size: 28px;left:26vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '><strong>Evaluation of NHM {subbasin} Domain Streamflow </strong><br><h1 style='position:absolute;z-index:100000;font-size: 20px;left:31vw;right:5vw; top:4vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '> {explan_txt}</h1>"

m.get_root().html.add_child(Element(title_html))

map_file = f"{html_maps_dir}/streamflow_map.html"
m.save(map_file)

webbrowser.open(map_file)

### Paste the poi_id in the field below

In [ ]:
poi_id_sel = None
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]

In [ ]:
v = widgets.Combobox(
    # value='John',
    placeholder="(optional) Enter Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
)


def on_change(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v.value


v.observe(on_change)

display(v)

In [ ]:
# This is for testing only; can comment out in user version
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[1]

In [ ]:
plot_file_path = create_streamflow_plot(
    obs_annual,
    obs,
    obs_efc,
    poi_id_sel,
    water_years,
    var_daily,
    var_units,
    poi_name_df,
    html_plots_dir,
    Folium_maps_dir,
)